In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
california_housing = fetch_california_housing(as_frame=True)
df = california_housing.frame

x = df.drop(['MedHouseVal'],axis=1)
y = df['MedHouseVal']

df.info()
df.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


MedInc         12928
HouseAge          52
AveRooms       19392
AveBedrms      14233
Population      3888
AveOccup       18841
Latitude         862
Longitude        844
MedHouseVal     3842
dtype: int64

In [3]:

X_train, X_test, y_train, y_test = train_test_split(x,y,train_size=0.8,random_state=42)

scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.transform(X_test)
X_test_scaler.shape,X_train_scaler.shape

((4128, 8), (16512, 8))

In [4]:
# اموزش مدل
model = LinearRegression()
model.fit(X_train_scaler, y_train)   
print(f"model intercept : {model.intercept_}")
print(f"model conf : {model.coef_}")

model intercept : 2.071946937378881
model conf : [ 0.85438303  0.12254624 -0.29441013  0.33925949 -0.00230772 -0.0408291
 -0.89692888 -0.86984178]


In [5]:
y_pred_train=model.predict(X_train_scaler)
r2_train = r2_score(y_train, y_pred_train)
print(f"R2 train = {r2_train:.4f}")

y_pred_test=model.predict(X_test_scaler)
r2_test = r2_score(y_test, y_pred_test)
print(f"R2 test = {r2_test:.4f}")

r2_train - r2_test

R2 train = 0.6126
R2 test = 0.5758


0.03676348536424445

In [6]:
mse_train = mean_squared_error(y_train,y_pred_train)
mse_test = mean_squared_error(y_test,y_pred_test)
mse_train,mse_test

(0.5179331255246699, 0.5558915986952444)

In [7]:
coef_df =pd.DataFrame ({
    'Feature': x.columns,
    'Coefficient' : model.coef_
})

coef_df = coef_df.sort_values(by='Coefficient', ascending=False)
print(coef_df)

max_positive_feature = coef_df.loc[coef_df['Coefficient'].idxmax(), 'Feature']
max_positive_coef = coef_df['Coefficient'].max()
print(f"بیشترین تأثیر مثبت: {max_positive_feature}\n با ضریب : {max_positive_coef:.4f}")

max_positive_feature = coef_df.loc[coef_df['Coefficient'].idxmin(), 'Feature']
max_positive_coef = coef_df['Coefficient'].min()
print(f"بیشترین تأثیر منفی: {max_positive_feature}\n با ضریب : {max_positive_coef:.4f}")


      Feature  Coefficient
0      MedInc     0.854383
3   AveBedrms     0.339259
1    HouseAge     0.122546
4  Population    -0.002308
5    AveOccup    -0.040829
2    AveRooms    -0.294410
7   Longitude    -0.869842
6    Latitude    -0.896929
بیشترین تأثیر مثبت: MedInc
 با ضریب : 0.8544
بیشترین تأثیر منفی: Latitude
 با ضریب : -0.8969


In [8]:
elastic = ElasticNet(alpha=0.1,l1_ratio=0.5)
elastic.fit(X_train_scaler, y_train)

y_pred_train_el=elastic.predict(X_train_scaler)
r2_train_el = r2_score(y_train, y_pred_train_el)
print(f"Elastic R2 train = {r2_train_el:.4f}")

y_pred_tes_el=elastic.predict(X_test_scaler)
r2_test_el = r2_score(y_test, y_pred_test)
print(f"Elastic R2 test = {r2_test_el:.4f}")

Elastic R2 train = 0.5308
Elastic R2 test = 0.5758


In [9]:
coef_df = pd.DataFrame({
    'Feature': x.columns,
    'Linear Coefficient': model.coef_,  # model = LinearRegression
    'Elastic Net Coefficient': elastic.coef_,
    'Difference': model.coef_ - elastic.coef_
}).round(4)

coef_df = coef_df.sort_values(by='Linear Coefficient', ascending=False)
print(coef_df)

      Feature  Linear Coefficient  Elastic Net Coefficient  Difference
0      MedInc              0.8544                   0.7138      0.1406
3   AveBedrms              0.3393                   0.0000      0.3393
1    HouseAge              0.1225                   0.1373     -0.0147
4  Population             -0.0023                   0.0000     -0.0023
5    AveOccup             -0.0408                  -0.0000     -0.0408
2    AveRooms             -0.2944                  -0.0000     -0.2944
7   Longitude             -0.8698                  -0.1333     -0.7366
6    Latitude             -0.8969                  -0.1757     -0.7212


In [10]:
pca = PCA(n_components=5)
x_train_pca = pca.fit_transform(X_train_scaler)
x_test_pca = pca.transform(X_test_scaler)


In [11]:
model_pca = LinearRegression()
model_pca.fit(x_train_pca, y_train)


LinearRegression()

In [12]:
y_pred_train_pca=model_pca.predict(x_train_pca)
r2_train_pca = r2_score(y_train, y_pred_train_pca)
print(f"pca R2 train = {r2_train_pca:.4f}")

y_pred_test_pca=model_pca.predict(x_test_pca)
r2_test_pca = r2_score(y_test, y_pred_test_pca)
print(f"pca R2 test = {r2_test_pca:.4f}")

print(f"Difference (Overfitting Index): {abs(r2_train_pca - r2_test_pca):.4f}")

pca R2 train = 0.4567
pca R2 test = 0.4329
Difference (Overfitting Index): 0.0238


In [13]:
comparison_df = pd.DataFrame({
    'Model': [
        'Linear Regression',
        'Elastic Net',
        'Linear Regression + PCA'
    ],
    'Train R2': [
        r2_train,
        r2_train_el,
        r2_train_pca
    ],
    'Test R2': [
        r2_test,
        r2_test_el,
        r2_test_pca
    ]
})

comparison_df['Overfitting Gap'] = (
    comparison_df['Train R2'] - comparison_df['Test R2']
).abs()

comparison_df = comparison_df.round(4)

comparison_df

Model  Train R2  Test R2  Overfitting Gap
0        Linear Regression    0.6126   0.5758           0.0368
1              Elastic Net    0.5308   0.5758           0.0450
2  Linear Regression + PCA    0.4567   0.4329           0.0238